In [1]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
import astrodendro
import os
from astropy import wcs
from astrodendro import Dendrogram, pp_catalog
from astrodendro.analysis import PPStatistic
from astropy.table import Table, hstack, Column
from astropy.utils.console import ProgressBar
import reproject
import pyregion
print(np.__version__)
import aplpy

1.16.5


In [2]:
# Image information and assumptions
distance        = 8340. # distance to GC; Reid et al. 2014
#Temp            = 20.0
Wave            = (3.0e8/226.0e9)
Wave0           = 1.3e-3
k0              = 0.899
nu              = 3.e08/Wave
nu0             = 3.e08/Wave0
beta            = 1.75
Kappag2d        = k0*((nu/nu0)**beta)
g2d             = 100.0
Kappa           = Kappag2d / g2d
mu              = 2.8 # express everything in H2

dustTemp = 20

# Constants
G = 6.67408e-11
msun = 1.989e33
mh = 1.6737236e-27
pc2cm = 3.08567758e18
as2persr = 4.25e10 
percm2perm = 1.0e6
JyperSr_to_JyperPix = (3.0462*(10**-4))*((0.00013888888888)**2)
JyperSr_to_Jyperassqr = ((np.pi*180.)**(-2))*(3600**(-1))
hplanck = 6.63e-34
clight = 2.99792e8
kboltzmann = 1.381e-23
sin1yr = 3.15569e7
arcsec2pc = distance/((360./(2.*np.pi))*60.*60.)

### Define physical property functions

def planck_wave( Wave, Temp ):

    # Planck function using wavelength
    #output in Jy/Sr I think?

    planck_conv_wave = 1.e-26 * clight / Wave**2.0

    planck = ((2.0*hplanck*clight**2.0)/(Wave**5.0))*(1.0/(np.exp((hplanck*clight)/(Wave*kboltzmann*Temp))-1.0))
    planck = planck/planck_conv_wave

    return planck

def mass_calc_submm( Wave, Temp, Kappa, Integrated_Flux, Obj_Dist ):
    
    Obj_Dist = Obj_Dist * pc2cm
    B = planck_wave( Wave, Temp )
    Mass = (Obj_Dist**2. * Integrated_Flux) / ( Kappa * B )
    Mass = Mass / msun
    return Mass

def column_density(Wave, Temp, Kappa, Flux, mu):
    
    B = planck_wave( Wave, Temp )
    N = Flux / (mu * (mh*1.e3) * Kappa * B)
    return N

def number_density_sphere_pc( Mass_sol, Radius_pc, mu ):

    # This subroutine accepts mass in solar masses and radius in pc and calculates the number density.

    Mass = Mass_sol * (msun/1000.0)
    Radius = Radius_pc * (pc2cm/100.0)

    n = Mass / (((4. / 3.)*np.pi) * mu * mh * Radius**3.0)

    # Convert to particles per cubic centimetre

    n = n / percm2perm

    return n

def mass_density_sphere( Mass_sol, Radius_pc ):

    # This subroutine accepts mass in solar masses and radius in pc and calculates the mass density.

    Mass = Mass_sol * (msun/1000.0)
    Radius = Radius_pc * (pc2cm/100.0)

    rho = Mass / (((4. / 3.)*np.pi) * Radius**3.0)

    return rho

def tff_spherical_F( number_density, mu ):

    # Accepts a number density in units of particles per cubic centimetre and derives the free fall time in yrs

    mass_density = mu * mh * number_density * percm2perm

    tff = np.sqrt( (3. * np.pi) / (32. * G * mass_density) )

    tff = tff / sin1yr # free-fall time in years

    return tff

### Takes mass in solar masses and volume density constant in 1/cm^3 and gives radius in parsec.
def radius_from_mass_const_vol_density(Mass_solar,vol_dens):
    #print Mass_solar
    Mass = Mass_solar * (msun/1000.0)
    #print Mass
    vol_dens = vol_dens *  percm2perm
    #print vol_dens
    Radius_cubed = (Mass / (mu * mh * vol_dens)) / ((4. / 3.)*np.pi)
    #print Radius_cubed
    Radius_cm = Radius_cubed**(1./3.)
    #print Radius_cm
    Radius_pc = Radius_cm / (pc2cm/100)
    return Radius_pc


def radius_from_mass_const_col_density(Mass_solar,col_dens):
    #print Mass_solar
    Mass = Mass_solar * (msun/1000.0)
    #print Mass
    col_dens = col_dens * (10**4) #convert from cm^-2 to m^-2

    Radius_sqrd = (Mass / (mu * mh * col_dens)) / (np.pi)
    #print Radius_cubed
    Radius_cm = Radius_sqrd**(1./2.)
    #print Radius_cm
    Radius_pc = Radius_cm / (pc2cm/100)
    return Radius_pc
   
    
def mass_from_radius_const_col_density(Radius_pc,col_dens):
    Radius_cm = Radius_pc * (pc2cm)#/100) 
    Radius_sqrd = Radius_cm**2
    Mass = Radius_sqrd * (np.pi) * (mu * mh * col_dens)
    col_dens = col_dens * (10**4) #convert from cm^-2 to m^-2
    Mass_solar = Mass / (msun/1000.0)
    return Mass_solar



def mass_err(I_int, I_peak_err, Td, Td_err, d, d_err, area, Kappa):
    '''
    I_int: integrated flux in Janskys
    I_peak_err: The peak flux uncertainty in Janskys per Steradian
    Td: dust temperature in Kelvin
    Td_err: dust temperature uncertainty in Kelvin
    d: distance to galactic center in parsecs
    d_err: distance uncertainty in parsecs
    npix: size of object in square arcsec
    Kappa: assumed dust opacity in cm^2 /gram
    '''
    d = d * pc2cm
    d_err = d_err * pc2cm
    I_int_err = I_peak_err*(area/(3600*3600*3282.8)) #convert area to sr 
    B = planck_wave(Wave,Td)
    dterm = (2*d*I_int*d_err)/(Kappa*B)
    Iterm = (I_int_err*d**2)/(Kappa*B)
    Tterm = Td_err * ((I_int * d**2)/Kappa) * (((1.0e-26)*(Wave**2))/(2*kboltzmann*(Td**2))) *np.exp(hplanck*clight/(kboltzmann*Wave*Td))
    return np.sqrt(dterm**2 + Iterm**2 + Tterm**2)/msun


def N_err(I_peak, I_peak_err, Td, Td_err, Kappa):
    '''
    I_peak: peak flux in Janskys
    I_peak_err: The peak flux uncertainty in Janskys per Steradian
    Td: dust temperature in Kelvin
    Td_err: dust temperature uncertainty in Kelvin
    Kappa: assumed dust opacity in cm^2 /gram
    '''
    B = planck_wave(Wave,Td)
    Iterm = (I_peak_err)/(mu * (mh*1.e3) * Kappa * B)
    Tterm = Td_err * (I_peak/(mu * (mh*1.e3) * Kappa)) * (((1.0e-26)*(Wave**2))/(2*kboltzmann*(Td**2))) *np.exp(hplanck*clight/(kboltzmann*Wave*Td))
    return np.sqrt(Iterm**2 + Tterm**2)

def ndens_err(I_int, I_peak_err, Td, Td_err, d, d_err, area, Kappa, Radius_pc):
    """
    I_int: integrated flux in Janskys
    I_peak_err: The peak flux uncertainty in Janskys per Steradian
    Td: dust temperature in Kelvin
    Td_err: dust temperature uncertainty in Kelvin
    d: distance to galactic center in parsecs
    d_err: distance uncertainty in parsecs
    npix: size of object in square arcsec
    Kappa: assumed dust opacity in cm^2 /gram
    Radius_pc: effective radius in parsecs
    """
    Radius = Radius_pc * (pc2cm/100.0) #convert to SI
    m_err = mass_err(I_int=catalog['flux_integrated'],I_peak_err=catalog['noise'],
                     Td=20.0,Td_err=5.0,
                     d=8150.0,d_err=150.0,
                     area=catalog['area_exact'],Kappa=Kappa)
    Mass = m_err*msun / (1000.0) #convert to SI
    n_err = Mass/(((4./3.)*np.pi) * mu * mh * Radius**3.0)
    n_err = n_err / percm2perm
    return n_err

def rho_err(I_int, I_peak_err, Td, Td_err, d, d_err, area, Kappa, Radius_pc):
    """
    I_int: integrated flux in Janskys
    I_peak_err: The peak flux uncertainty in Janskys per Steradian
    Td: dust temperature in Kelvin
    Td_err: dust temperature uncertainty in Kelvin
    d: distance to galactic center in parsecs
    d_err: distance uncertainty in parsecs
    npix: size of object in square arcsec
    Kappa: assumed dust opacity in cm^2 /gram
    Radius_pc: effective radius in parsecs
    returns mass density in 
    """
    Radius = Radius_pc * (pc2cm/100.0) #convert to SI
    m_err = mass_err(I_int=catalog['flux_integrated'],I_peak_err=catalog['noise'],
                     Td=20.0,Td_err=5.0,
                     d=8150.0,d_err=150.0,
                     area=catalog['area_exact'],Kappa=Kappa)
    Mass = m_err*msun / (1000.0) #convert to cgs
    rho_err = Mass/(((4./3.)*np.pi) * Radius**3.0)

    return rho_err
    
def tff_err(I_int, I_peak_err, Td, Td_err, d, d_err, area, Kappa, Radius_pc, mass_density):
    """
    I_int: integrated flux in Janskys
    I_peak_err: The peak flux uncertainty in Janskys per Steradian
    Td: dust temperature in Kelvin
    Td_err: dust temperature uncertainty in Kelvin
    d: distance to galactic center in parsecs
    d_err: distance uncertainty in parsecs
    npix: size of object in square arcsec
    Kappa: assumed dust opacity in cm^2 /gram
    Radius_pc: effective radius in parsecs
    returns freefall time error in years
    """
    number_density_err = ndens_err(I_int, I_peak_err, Td, Td_err, d, d_err, area, Kappa, Radius_pc)
    mass_density_err = mu * mh * number_density_err * percm2perm
    sigma_tff = np.sqrt((3. * np.pi) / (32. * G * mass_density**3))*mass_density_err
    sigma_tff = sigma_tff / sin1yr # free-fall time in years

    return sigma_tff

